<a href="https://colab.research.google.com/github/machren/NNQuantumStates/blob/main/QIC_TFI_DMRG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installing requirements for tenpy

In [1]:
!git clone https://github.com/tenpy/tenpy.git

fatal: destination path 'tenpy' already exists and is not an empty directory.


In [2]:
%cd tenpy

/content/tenpy


In [3]:
!pip install -e .

Obtaining file:///content/tenpy
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for physics-tenpy (pyproject.toml) ... done
  Created wheel for physics-tenpy: filename=physics_tenpy-1.0.5-0.editable-cp311-cp311-linux_x86_64.whl size=11706 sha256=82309311c502ec2fb3ec6b5c9dc9b0ce86869fe56797d3b5f064bf05a71a46b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-lted7jw7/wheels/bd/85/a6/9b2733a9d2d26a605422357d23565058a3dd71aca54fbe1d9e
Successfully built physics-tenpy
  Attempting uninstall: physics-tenpy
    Found existing installation: physics-tenpy 1.0.6
    Uninstalling physics-tenpy-1.0.6:
      Successfully uninstalled physics-tenpy-1.0.6


In [4]:
import tenpy
print(tenpy.__version__)


1.0.5


### Transverse Field Ising Model and DMRG in TeNPy

Here we describe implementation of the transverse field Ising model (TFIModel) using the Tensor Network Python library (TeNPy), focusing on its Hamiltonian structure, model parameters, DMRG algorithm settings, and the theoretical background based on the matrix product state (MPS) framework introduced in [Schollwöck (2011)](https://arxiv.org/abs/1008.3477).

---

#### Transverse Field Ising Model

The TFIM is defined on a lattice with the Hamiltonian consisting of nearest-neighbor spin interactions and a transverse magnetic field. The model captures the competition between ordering (due to spin-spin coupling) and quantum fluctuations (due to the transverse field).

#### Model Parameters

The model is specified via a configuration dictionary. The values in the example are selected for physical and numerical reasons:

- **`L = N`** – System size, defining the number of lattice sites. Typical choices range from 10 to 100 depending on available computational resources. Larger sizes approach the thermodynamic limit but increase computational cost.

- **`J = 1.0`** – The spin-spin coupling constant. Setting \( J = 1 \) is a standard normalization choice that allows comparison across different simulations. Other parameters (like `g`) are then interpreted relative to this unit.

- **`g = 1.0`** – The strength of the transverse magnetic field. The value \( g = 1 \) is particularly significant, as it represents the critical point of the 1D transverse field Ising model in the thermodynamic limit. At this value, the system undergoes a quantum phase transition from a ferromagnetic to a paramagnetic phase.

- **`bc_MPS = 'finite'`** – Boundary conditions applied to the MPS representation. 'Finite' implies open boundary conditions, suitable for systems of finite size. They are easier to handle numerically than periodic conditions.

---

#### DMRG Algorithm Parameters

TeNPy’s DMRG implementation is highly customizable. The parameters selected in the example balance precision, convergence reliability, and computational efficiency.

#### Algorithm Parameters

- **`mixer = True`** – Enables the use of a mixer, which introduces noise into the local optimization steps during early sweeps. This helps prevent the algorithm from getting stuck in local minima. It is gradually turned off, allowing fine convergence near the true ground state.

- **`max_E_err = 1e-10`** – Convergence threshold for the energy. The DMRG iterations stop when the energy changes between sweeps fall below this value. The choice of \(10^{-10}\) ensures highly accurate energy estimates suitable for precision studies, albeit with longer computation times.

- **`trunc_params`**:
  - **`chi_max = 1000`** – The maximum number of states (bond dimension) retained during truncation. A value of 1000 strikes a balance between accuracy and memory usage. Larger `chi_max` allows for more entanglement to be captured but increases computational cost.
  - **`svd_min = 1e-10`** – Minimum singular value threshold. Singular values smaller than this are discarded. This cutoff ensures numerical stability and avoids keeping noise-level components in the MPS.

- **`verbose = 1`** – Controls the level of output. A value of 1 provides moderate logging of DMRG progress without overwhelming detail, useful for monitoring convergence in typical simulations.

- **`N_sweeps_check = 2`** – The number of sweeps between convergence checks. Frequent checks help terminate the algorithm promptly when convergence is reached, saving computational time.

---

#### DMRG Implementation in TeNPy

TeNPy implements the Density Matrix Renormalization Group (DMRG) based on the modern formulation using Matrix Product States, as developed in [Schollwöck (2011)](https://arxiv.org/abs/1008.3477).

#### Key Features of the MPS-Based DMRG Algorithm

1. **Matrix Product States (MPS)**  
   Quantum many-body wavefunctions are approximated as a product of local tensors. This efficiently encodes entanglement for gapped 1D systems, leveraging the area law.

2. **Variational Optimization**  
   DMRG iteratively optimizes the MPS by minimizing the system's energy, sweeping back and forth over the chain. At each step, tensors are updated locally based on their effective Hamiltonian.

3. **Two-Site and One-Site Updates**  
   The algorithm typically begins with two-site updates to ensure robustness and transitions to faster one-site updates as convergence improves. The transition is guided by convergence metrics and the `mixer` mechanism.

4. **Truncation and SVD**  
   After optimization, tensors are decomposed using Singular Value Decomposition. The MPS is truncated to retain only the most significant singular values (based on `chi_max` and `svd_min`) to control the growth of computational complexity.

5. **Convergence and Stopping Criteria**  
   Energy convergence is monitored based on `max_E_err` and checked every `N_sweeps_check` sweeps. Once changes in energy become negligible, the algorithm halts, yielding the ground-state MPS.

---

## References

- Schollwöck, U. (2011). *The density-matrix renormalization group in the age of matrix product states*. [arXiv:1008.3477](https://arxiv.org/abs/1008.3477)


In [19]:
import numpy as np
import time
from tenpy.models import TFIModel
from tenpy.models.tf_ising import TFIChain
from tenpy.networks.mps import MPS
from tenpy.algorithms import dmrg, dmrg_parallel

In [15]:
import scipy.sparse as sparse
from scipy.sparse.linalg import eigsh
import warnings
import scipy.integrate

def finite_gs_energy(L, J, g):
    """For comparison: obtain ground state energy from exact diagonalization.

    Exponentially expensive in L, only works for small enough `L` <~ 20.
    """
    if L >= 20:
        warnings.warn("Large L: Exact diagonalization might take a long time!")
    # get single site operaors
    sx = sparse.csr_matrix(np.array([[0., 1.], [1., 0.]]))
    sz = sparse.csr_matrix(np.array([[1., 0.], [0., -1.]]))
    id = sparse.csr_matrix(np.eye(2))
    sx_list = []  # sx_list[i] = kron([id, id, ..., id, sx, id, .... id])
    sz_list = []
    for i_site in range(L):
        x_ops = [id] * L
        z_ops = [id] * L
        x_ops[i_site] = sx
        z_ops[i_site] = sz
        X = x_ops[0]
        Z = z_ops[0]
        for j in range(1, L):
            X = sparse.kron(X, x_ops[j], 'csr')
            Z = sparse.kron(Z, z_ops[j], 'csr')
        sx_list.append(X)
        sz_list.append(Z)
    H_xx = sparse.csr_matrix((2**L, 2**L))
    H_z = sparse.csr_matrix((2**L, 2**L))
    for i in range(L - 1):
        H_xx = H_xx + sx_list[i] * sx_list[(i + 1) % L]
    for i in range(L):
        H_z = H_z + sz_list[i]
    H = -J * H_xx - g * H_z
    E, V = eigsh(H, k=1, which='SA', return_eigenvectors=True, ncv=20)
    return E[0]

def infinite_gs_energy(J, g):
    """For comparison: Calculate groundstate energy density from analytic formula.

    The analytic formula stems from mapping the model to free fermions, see P. Pfeuty, The one-
    dimensional Ising model with a transverse field, Annals of Physics 57, p. 79 (1970). Note that
    we use Pauli matrices compared this reference using spin-1/2 matrices and replace the sum_k ->
    integral dk/2pi to obtain the result in the N -> infinity limit.
    """
    def f(k, lambda_):
        return np.sqrt(1 + lambda_**2 + 2 * lambda_ * np.cos(k))

    E0_exact = -g / (J * 2. * np.pi) * scipy.integrate.quad(f, -np.pi, np.pi, args=(J / g, ))[0]
    return E0_exact

In [13]:
# DMRG parameters
dmrg_params = {
    'mixer': True,            # to avoid that the algorithm gets stuck in local energy minima
    'max_E_err': 1e-10,
    'trunc_params': {
        'chi_max': 1000,
        'svd_min': 1e-10,
    },
    'verbose': 1,
    'N_sweeps_check': 2,
}

ground_energies = []
times = []

In [14]:
for N in range(10, 100, 10):
  # Define model parameters
  model_params = {
      'L': N,                   # system size
      'J': 1.,                 # coupling constant
      'g': 1.,                 # transverse field
      'bc_MPS': 'finite',       # boundary conditions
  }

  # Initialize the model
  model = TFIModel(model_params)

  # Build the initial product state (all spins up)
  product_state = ["up"] * model.lat.N_sites

  psi = MPS.from_product_state(model.lat.mps_sites(), product_state, bc=model_params['bc_MPS'])

  # Run the DMRG algorithm
  start_time = time.time()
  info = dmrg.run(psi, model, dmrg_params)
  end_time = time.time()

  times.append(end_time - start_time)
  ground_energies.append(info['E'])

  print("final bond dimensions: ", psi.chi)
  mag_x = np.sum(psi.expectation_value("Sigmax"))
  mag_z = np.sum(psi.expectation_value("Sigmaz"))
  print("magnetization in X = {mag_x:.5f}".format(mag_x=mag_x))
  print("magnetization in Z = {mag_z:.5f}".format(mag_z=mag_z))
  if N < 20:  # compare to exact result
      E_exact = finite_gs_energy(N, 1., 1.)
      print("Exact diagonalization: E = {E:.13f}".format(E=E_exact))
      print("relative error: ", abs((info['E'] - E_exact) / E_exact))

  print(f"N = {N}: Ground state energy = {info['E']}")
  print(f"Time taken is {end_time - start_time} seconds")

/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


final bond dimensions:  [2, 4, 8, 14, 19, 14, 8, 4, 2]
magnetization in X = 0.00000
magnetization in Z = 7.32255
Exact diagonalization: E = -12.3814899996547
relative error:  4.3040623691892936e-16
N = 10: Ground state energy = -12.381489999654743
Time taken is 1.8805060386657715 seconds


/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


final bond dimensions:  [2, 4, 8, 16, 22, 27, 30, 33, 33, 33, 33, 33, 30, 27, 22, 16, 8, 4, 2]
magnetization in X = 0.00000
magnetization in Z = 13.89885
N = 20: Ground state energy = -25.107797111623743
Time taken is 5.991666316986084 seconds


/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


...

/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


final bond dimensions:  [2, 4, 8, 16, 22, 30, 34, 36, 40, 44, 45, 47, 50, 52, 56, 57, 60, 61, 62, 64, 64, 66, 67, 68, 69, 70, 71, 72, 72, 73, 73, 73, 73, 74, 74, 74, 74, 74, 73, 73, 73, 72, 71, 70, 69, 68, 67, 66, 64, 64, 62, 61, 61, 58, 56, 52, 50, 47, 45, 44, 40, 36, 34, 30, 22, 16, 8, 4, 2]
magnetization in X = 0.00000
magnetization in Z = 46.12081
N = 70: Ground state energy = -88.76524466396042
Time taken is 173.0730230808258 seconds


/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


final bond dimensions:  [2, 4, 8, 16, 22, 30, 34, 36, 40, 45, 45, 47, 50, 52, 57, 58, 60, 62, 62, 64, 65, 67, 69, 70, 72, 73, 74, 76, 77, 77, 77, 78, 78, 78, 80, 80, 80, 81, 80, 80, 80, 81, 80, 80, 80, 79, 78, 78, 78, 77, 77, 76, 74, 73, 72, 71, 69, 67, 65, 64, 63, 62, 60, 57, 57, 52, 50, 47, 46, 45, 40, 36, 34, 30, 22, 16, 8, 4, 2]
magnetization in X = 0.00000
magnetization in Z = 52.52911
N = 80: Ground state energy = -101.4974094523919
Time taken is 301.6493170261383 seconds
final bond dimensions:  [2, 4, 8, 16, 22, 30, 34, 36, 40, 45, 47, 47, 50, 52, 57, 58, 61, 62, 64, 65, 67, 68, 70, 72, 74, 76, 77, 77, 78, 79, 80, 80, 81, 82, 83, 83, 83, 83, 83, 83, 83, 84, 84, 85, 85, 85, 85, 85, 84, 84, 83, 83, 83, 83, 83, 82, 81, 81, 80, 79, 78, 77, 77, 76, 74, 72, 70, 68, 67, 65, 64, 62, 61, 58, 57, 53, 50, 48, 47, 45, 41, 36, 34, 30, 22, 16, 8, 4, 2]
magnetization in X = 0.00000
magnetization in Z = 58.93249
N = 90: Ground state energy = -114.22962521669884
Time taken is 391.6656262874603 s

/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [18]:
# Print ground state energy
print("Ground state energy:", ground_energies)

Ground state energy: [np.float64(-12.381489999654743), np.float64(-25.107797111623743), np.float64(-37.838098239709325), np.float64(-50.56943379479517), np.float64(-63.30118915542022), np.float64(-76.03315613032177), np.float64(-88.76524466396042), np.float64(-101.4974094523919), np.float64(-114.22962521669884)]


### Single-site infinite DMRG, transverse field Ising model


In [17]:
# iDMRG parameters
idmrg_params = {
    'mixer': True,            # to avoid that the algorithm gets stuck in local energy minima
    'max_E_err': 1e-10,
    'trunc_params': {
        'chi_max': 30,
        'svd_min': 1e-10,
    },
    'verbose': 1,
    'N_sweeps_check': 2,
}

model_params = dict(L=2, J=1., g=1., bc_MPS='infinite', conserve=None)

M = TFIChain(model_params)

product_state = ["up"] * M.lat.N_sites

psi = MPS.from_product_state(M.lat.mps_sites(), product_state, bc=M.lat.bc_MPS)

eng = dmrg.SingleSiteDMRGEngine(psi, M, idmrg_params)
E, psi = eng.run()  # equivalent to dmrg.run() up to the return parameters.

print("E = {E:.13f}".format(E=E))
print("final bond dimensions: ", psi.chi)

mag_x = np.mean(psi.expectation_value("Sigmax"))
mag_z = np.mean(psi.expectation_value("Sigmaz"))

print("<sigma_x> = {mag_x:.5f}".format(mag_x=mag_x))
print("<sigma_z> = {mag_z:.5f}".format(mag_z=mag_z))
print("correlation length:", psi.correlation_length())

# compare to exact result
E_exact = infinite_gs_energy(1., 1.)
print("Analytic result: E (per site) = {E:.13f}".format(E=E_exact))
print("relative error: ", abs((E - E_exact) / E_exact))

E = -1.9099013180437
final bond dimensions:  [30, 30]
<sigma_x> = -0.29321
<sigma_z> = 0.63662
correlation length: 790.001203244766
Analytic result: E (per site) = -1.2732395447352
relative error:  0.5000329874618831
